In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

In [22]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import pandas as pd
import sqlite3
print(os.getcwd())
cnx = sqlite3.connect('../data.db')
import numpy as np

/Users/naeeramin/Documents/UTS_3rd_semester/ilab2/github_repo/ARDC-Project/notebooks


In [21]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.services.LinkedInDataService import LinkedInDataService
linkedInDataService = LinkedInDataService(cnx)
lobbyist_employee_with_linkedin_profile_identification = linkedInDataService.get_all_lobbyist_employees_with_profile_identification_stats()

In [23]:
# print the column names
lobbyist_employee_with_linkedin_profile_identification.columns


Index(['name', 'current_company', 'profile_title', 'sub_title', 'university',
       'correct_name', 'correct_company', 'worked_for_gov'],
      dtype='object')

In [25]:
# make a copy of the dataframe
data = lobbyist_employee_with_linkedin_profile_identification
totalEmployees = 0 
correctProfileScraped = 0
workForGov = 0
for i in range(len(data)):
    totalEmployees += 1
    if data.loc[i, "correct_name"] == 1 and data.loc[i, "correct_company"] == 1:
        correctProfileScraped += 1
        if data.loc[i, "worked_for_gov"] == 1:
            workForGov += 1

print(totalEmployees)
print(correctProfileScraped)
print(workForGov)

print("Percentage of Correct Profiles Scraped: " + str((correctProfileScraped/totalEmployees)*100))
print("Percentage of Employees with Government Experience from correct profiles scraped: " + str((workForGov/correctProfileScraped)*100))


237
116
88
Percentage of Correct Profiles Scraped: 48.9451476793249
Percentage of Employees with Government Experience from correct profiles scraped: 75.86206896551724


In [5]:
pd.options.display.max_colwidth = 100
sql = """
WITH gov_positions_bins as (
	WITH num_gov_positions_held as(
		WITH employee_gov_positions as(
			SELECT  DISTINCT lp2.name, count(DISTINCT lp2.role) as num_positions
			FROM linkedin_profile_info lp1
			INNER join linkedin_user_work_experience  lp2 on lp1.id = lp2.profile_id
			INNER join linkedin_user_profile_identification lp3 on lp1.id = lp3.profile_id
			WHERE lp2.government_experience = 1 AND lp3.correct_name = 1 AND lp3.correct_company = 1
			GROUP BY lp2.name
		)
		SELECT num_positions as num_gov_positions_held, count(*) as num_employees
		FROM employee_gov_positions
		GROUP BY num_positions
	)
	SELECT num_gov_positions_held, num_employees,
	CASE
		WHEN num_gov_positions_held <2 THEN '<2'
		WHEN num_gov_positions_held >=2 AND num_gov_positions_held <=4 THEN '2-4'
		WHEN num_gov_positions_held >=5 AND num_gov_positions_held <=7 THEN '5-7'
		WHEN num_gov_positions_held >7 THEN '>7'
	END num_gov_positions_bins
	FROM num_gov_positions_held
)
SELECT num_gov_positions_bins, sum(num_employees) as num_employees
FROM gov_positions_bins
GROUP BY num_gov_positions_bins
"""
df = pd.read_sql(sql, cnx)
df.to_csv("../num_gov_positions.csv", index=False)
df

,num_gov_positions_bins,num_employees
0,2-4,43
1,5-7,5
2,<2,29
3,>7,3


In [66]:
pd.options.display.max_colwidth = 500
sql = """
SELECT DISTINCT lp1.name, lp2.company_name, lp2.role, lp2.duration
FROM linkedin_profile_info lp1
INNER join linkedin_user_work_experience  lp2 on lp1.id = lp2.profile_id
INNER join linkedin_user_profile_identification lp3 on lp1.id = lp3.profile_id
WHERE lp2.government_experience = 1 AND lp3.correct_name = 1 AND lp3.correct_company = 1
AND lp1.name = 'Christopher Pyne'
"""
df = pd.read_sql(sql, cnx)
display(df)
df.to_csv("../ChristopherPyne.csv", index=False)


,name,company_name,role,duration
0,Christopher Pyne,Australian Government,"Minister for Defence, Leader of the House",Aug 2018 - May 2019 · 10 mos
1,Christopher Pyne,Australian Government,Federal Member for Sturt,Mar 1993 - May 2019 · 26 yrs 3 mos
2,Christopher Pyne,Australian Government,"Minister for Defence Industry, Leader of the House",Jul 2016 - Aug 2018 · 2 yrs 2 mos
3,Christopher Pyne,Australian Government,"Minister for Industry, Innovation and Science, Leader of the House",Sep 2015 - Jul 2016 · 11 mos
4,Christopher Pyne,Australian Government,"Minister for Education and Training, Leader of the House",Sep 2013 - Sep 2015 · 2 yrs 1 mo
5,Christopher Pyne,Australian Government,"Shadow Minister for Education, Apprenticeships & Training, Manager of Opposition Business",2009 - Sep 2013 · 4 yrs 9 mos
6,Christopher Pyne,Australian Government,Shadow Minister for Justice & Border Protection,Dec 2007 - Sep 2008 · 10 mos
7,Christopher Pyne,Australian Government,Minister for Ageing,Mar 2007 - Dec 2007 · 10 mos
8,Christopher Pyne,Australian Government,Assistant Minister for Health & Ageing,Jan 2007 - Mar 2007 · 3 mos
9,Christopher Pyne,Australian Government,Parliamentary Secretary for Health & Ageing,Oct 2004 - Jan 2007 · 2 yrs 4 mos


In [68]:
pd.options.display.max_colwidth = 500
sql = """
SELECT DISTINCT lp1.name as Name, lp2.company_name as OrganisationName, lp2.role as Role, lp2.duration as Duration
FROM linkedin_profile_info lp1
INNER join linkedin_user_work_experience  lp2 on lp1.id = lp2.profile_id
INNER join linkedin_user_profile_identification lp3 on lp1.id = lp3.profile_id
WHERE lp2.government_experience = 1 AND lp3.correct_name = 1 AND lp3.correct_company = 1
AND lp1.name = 'Louise Yabsley'
"""
df = pd.read_sql(sql, cnx)
df.to_csv("../LouiseYabsley.csv", index=False)
df

,Name,OrganisationName,Role,Duration
0,Louise Yabsley,Australian Government Department of Finance,"NDIS and Aged Care, Budget Group",Mar 2015 - Oct 2015 · 8 mos
1,Louise Yabsley,Australian Government Department of Finance,"Acting Assistant Director, Analysis and Projections",Nov 2013 - Mar 2015 · 1 yr 5 mos
2,Louise Yabsley,Australian Government Department of Finance,"Senior Policy Officer, Carbon Tax Repeal Taskforce",Aug 2013 - Nov 2013 · 4 mos
3,Louise Yabsley,Australian Government Department of Climate Change and Energy Efficiency,"Senior Policy Officer, Carbon Pricing and Markets Division",Sep 2012 - Jul 2013 · 11 mos
4,Louise Yabsley,Australian Government Department of Climate Change and Energy Efficiency,"Senior Policy Officer, International Division",Apr 2011 - Sep 2012 · 1 yr 6 mos
5,Louise Yabsley,Australian Government Department of Climate Change and Energy Efficiency,"Departmental Liaison Officer, Office of the Hon Greg Combet AM MP",Apr 2009 - Apr 2010 · 1 yr 1 mo
6,Louise Yabsley,Australian Government Department of Climate Change and Energy Efficiency,"Senior Policy Officer, Adaptation and Science Division",Aug 2008 - Apr 2009 · 9 mos
7,Louise Yabsley,Australian Government Department of Veterans' Affairs,"Programme Officer, Access Card Programme Office",Nov 2006 - Nov 2007 · 1 yr 1 mo
8,Louise Yabsley,Australian Government Department of Veterans' Affairs,Graduate,Feb 2006 - Nov 2006 · 10 mos


In [6]:
sql = """
SELECT DISTINCT lp1.name as Name, lp2.company_name as OrganisationName, lp2.role as Role, lp2.duration as Duration
FROM linkedin_profile_info lp1
INNER join linkedin_user_work_experience  lp2 on lp1.id = lp2.profile_id
INNER join linkedin_user_profile_identification lp3 on lp1.id = lp3.profile_id
WHERE lp2.government_experience = 1 AND lp3.correct_name = 1 AND lp3.correct_company = 1
"""
df = pd.read_sql(sql, cnx)
df

,Name,OrganisationName,Role,Duration
0,Julie Bishop,Australian Government,Member of House of Representatives,May 1998 - May 2019 · 21 yrs 1 mo
1,Julie Bishop,Australian Government,Minister for Foreign Affairs,Sep 2013 - Aug 2018 · 5 yrs
2,Julie Bishop,Australian Government,Deputy Leader of the Liberal Party of Australia,Nov 2007 - Aug 2018 · 10 yrs 10 mos
3,Julie Bishop,Australian Government,Minister for Education and Science,Jan 2006 - Dec 2007 · 2 yrs
4,Julie Bishop,Australian Government,Minister for Women,Jan 2006 - Dec 2007 · 2 yrs
...,...,...,...,...
214,Leonie M. Short,Parliament of Australia,Federal Member for Ryan,2001 · Less than a year
215,Tony Rolfe,"Jane Prentice MP, Assistant Minister for Disability Services",Policy Adviser,Apr 2016 - Aug 2017 · 1 yr 5 mos
216,Tony Rolfe,Dr Andrew Laming MP,Policy Adviser,Sep 2012 - Feb 2013 · 6 mos
217,Tony Rolfe,Shadow Minister for Mental Health & Ageing,Policy Adviser,May 2012 - Sep 2012 · 5 mos


In [86]:

df["Role"] = df["Role"].str.lower()
mp_senator_list = ['member', 'minister', 'deputy leader', 'leader', 'shadow minister', 'president',
'head of government', 'federal member', 'parliamentary secretary']
chief_staff_list = ['chief of staff', 'senior adviser', 'senior health adviser']
media_list = ['media adviser', 'media manager', 'media', 'press adviser', 'communications',
'press']
adviser_list = ['adviser', 'policy adviser']
positions_list = []
positions_list.append(df["Role"].str.contains('|'.join(media_list)).sum())
positions_list.append(df["Role"].str.contains('|'.join(adviser_list)).sum())
positions_list.append(df["Role"].str.contains('|'.join(chief_staff_list)).sum())
positions_list.append(df["Role"].str.contains('|'.join(mp_senator_list)).sum())
print(positions_list)
 
positions_stats = {
    'FormerPosition': ['Media adviser/Press secretary',
                        'Adviser', 'Chief of staff/senior adviser', 'Member of Parliament/Senator'],
    'Total': positions_list
}
print(positions_stats)
df2 = pd.DataFrame.from_dict(positions_stats)
print(df2)
df2.to_csv('../position_stats.csv', index=False)

[42, 78, 39, 75]
{'FormerPosition': ['Media adviser/Press secretary', 'Adviser', 'Chief of staff/senior adviser', 'Member of Parliament/Senator'], 'Total': [42, 78, 39, 75]}
                  FormerPosition  Total
0  Media adviser/Press secretary     42
1                        Adviser     78
2  Chief of staff/senior adviser     39
3   Member of Parliament/Senator     75
